In [6]:
# Bibliotheken importieren
import sys
import os
import pandas as pd
import numpy as np

# Pfad zum Projektverzeichnis hinzufügen
sys.path.append(os.path.abspath(".."))

# Verbindung zur Datenbank herstellen
from database.db_connection import DatabaseConnector

db_connector = DatabaseConnector()

print(database.db_connection.__file__)
print(dir(db_connector))

engine = db_connector.get_engine()

# SQL-Queries zum Laden der Daten
query_forecast = "SELECT * FROM historical_forecast;"
query_weather = "SELECT * FROM historical_weather;"

# Daten laden
forecast_df = pd.read_sql(query_forecast, engine)
weather_df = pd.read_sql(query_weather, engine)

# Überblick über die Forecast-Daten
print("=== Forecast DataFrame ===")
print(forecast_df.info())  # Struktur der Daten
print(forecast_df.head())  # Erste 5 Zeilen anzeigen

# Überblick über die Wetter-Daten
print("\n=== Weather DataFrame ===")
print(weather_df.info())  # Struktur der Daten
print(weather_df.head())  # Erste 5 Zeilen anzeigen


c:\Users\busin\Documents\GIT\dataanalyticsproject\dataanalyticsproject\database\db_connection.py
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'close_connection', 'connect', 'connection', 'database', 'get_connection', 'host', 'password', 'user']


AttributeError: 'DatabaseConnector' object has no attribute 'get_engine'